In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ShakespeareHW").getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/20 23:22:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Create an RDD from a text file (example)
rdd = sc.textFile("shakespeare.txt")

In [3]:
file_path = "/home/julianflemons8/hw0_Shakespeare_txt_spark/shakespeare.txt"  
shakespeare_text_rdd = sc.textFile(file_path)

In [4]:
# Split each line into words and convert to lowercase
words_lowercase_rdd = rdd.flatMap(lambda line: [word.lower() for word in line.split()])

# Collect the results
words_lowercase = words_lowercase_rdd.collect()

# Print the results
print(words_lowercase[:10])


['first', 'citizen:', 'before', 'we', 'proceed', 'any', 'further,', 'hear', 'me', 'speak.']


In [6]:
import re
#Remove punctuation
cleaned_words_rdd = words_lowercase_rdd.map(lambda word: re.sub(r'[^\w\s]', '', word))
cleaned_words_rdd = cleaned_words_rdd.filter(lambda word: word != '') # Remove any empty strings that might result

In [7]:
words_rdd = cleaned_words_rdd.flatMap(lambda line: line.split())

In [8]:
#Count word occurrences 
word_count = words_rdd.countByValue


In [9]:
# Count word occurrences
word_count = cleaned_words_rdd.countByValue()

# Convert the resulting dictionary to an RDD of (word, count) pairs
word_counts_rdd = sc.parallelize(list(word_count.items()))

In [10]:
# Sort by count in descending order
sorted_word_counts_rdd = word_counts_rdd.sortBy(lambda item: item[1], ascending=False)

# Take the top 10 words
top_10_words_no_preprocessing = sorted_word_counts_rdd.take(10)

# Print the results for Part 1
print("Top 10 frequent words (without preprocessing):")
for word, count in top_10_words_no_preprocessing:
    print(f"{word}: {count}")

Top 10 frequent words (without preprocessing):
the: 6283
and: 5680
to: 4766
i: 4653
of: 3757
you: 3142
my: 3118
a: 2987
that: 2569
in: 2362


In [11]:
# --- Part 3: Top 10 frequent words with stop word filtering ---
from nltk.corpus import stopwords
import nltk

try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

# Filter out stop words
filtered_words_rdd = cleaned_words_rdd.filter(lambda word: word not in stop_words)

# Count word occurrences after filtering stop words
word_counts_filtered = filtered_words_rdd.countByValue()

# Convert to RDD of (word, count) pairs
word_counts_filtered_rdd = sc.parallelize(list(word_counts_filtered.items()))

# Sort by count in descending order
sorted_word_counts_filtered_rdd = word_counts_filtered_rdd.sortBy(lambda item: item[1], ascending=False)

# Take the top 10 words after filtering
top_10_words_with_stopwords = sorted_word_counts_filtered_rdd.take(10)

# Print the results for Part 3
print("\nTop 10 frequent words (with stop word filtering):")
for word, count in top_10_words_with_stopwords:
    print(f"{word}: {count}")

# Stop Spark Session (moved here to be at the end of the script)
# spark.stop()


Top 10 frequent words (with stop word filtering):
thou: 1403
thy: 1059
king: 887
shall: 845
thee: 760
lord: 698
good: 669
come: 624
sir: 592
well: 563
